# MOS Stability

In [ ]:
import sys
sys.path.append("/work/b0990106x/trl/vc")
import importlib
import vc
importlib.reload(vc)
from NISQA.nisqa.NISQA_model import nisqaModel
import numpy as np
import json

In [ ]:
def get_reward(output_path, base_path):
    args_nisqa = {
        "mode": "predict_file",
        "pretrained_model": f"{base_path}/NISQA/weights/nisqa.tar",
        "deg": output_path,
        "data_dir": None,
        "output_dir": f"{base_path}/NISQA/result/",
        "csv_file": None,
        "csv_deg": None,
        "num_workers": 0,
        "bs": 1,
        "ms_channel": None,
    }
    args_nisqa["tr_bs_val"] = args_nisqa["bs"]
    args_nisqa["tr_num_workers"] = args_nisqa["num_workers"]
    nisqa = nisqaModel(args_nisqa)
    try:
        prediction = nisqa.predict()
        reward = float(prediction["mos_pred"].iloc[0])
        print("Reward:", reward)
        return reward
    except Exception as e:
        print("Error:", e)
        print("get_reward function end ___________________________")
        return None
    
def calculate_metrics(rewards):
    metrics = {
        "mean": np.mean(rewards),
        "median": np.median(rewards),
        "std_dev": np.std(rewards),
        "variance": np.var(rewards),
        "min": np.min(rewards),
        "max": np.max(rewards),
        "25th_percentile": np.percentile(rewards, 25),
        "75th_percentile": np.percentile(rewards, 75),
    }
    return metrics

def save_metrics(metrics, path,times, name):
    with open(f"{path}/{name}_{times}.json", "w") as f:
        json.dump(metrics, f, indent=4)

In [ ]:
# base_path = "/work/b0990106x/trl"

# output_path = f"{base_path}/mos_stability"
# audio_name = "example_save_1"
# audio_path= f"{base_path}/mos_stability/{audio_name}.wav"
# times = 10

# scores = []
# for i in range(times):
#     reward = get_reward(audio_path, base_path) # output_path is the audio file path
#     scores.append(reward)

# metrics = calculate_metrics(scores)
# save_metrics(metrics, output_path, times, audio_name) # output_path is the where we save metrics

# print(metrics)

# VC Model Stability

In [ ]:
from dpo_eval import eval_dpo, save_metrics, prepare_data, load_models_and_tokenizers, load_datasets, process_and_get_scores
import torch
import sys
import importlib
import torch
from transformers import BartForConditionalGeneration, AutoTokenizer
import numpy as np
import json
import os

sys.path.append("/work/b0990106x/trl/vc")
import vc
importlib.reload(vc)


In [ ]:
ar_checkpoint = "lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans"
nar_checkpoint = "lca0503/speech-chatgpt-base-nar-v2-epoch4-wotrans"
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_data_len = 10
# trained_model_name = ts # from model_output
# output_dir_name = ts # output audio
agent_input_dir = "/work/b0990106x/trl/data-encodec"
base_path = "/work/b0990106x/trl"

In [ ]:
def load_from_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    all_src_encodec = [item["src_encodec"] for item in data]
    all_instruction = [item["instruction"] for item in data]
    all_tgt_encodec = [item["tgt_encodec"] for item in data]
    
    return all_src_encodec, all_instruction, all_tgt_encodec

# Usage
all_src_encodec, all_instruction, all_tgt_encodec = load_from_json('src_encodec.json')

print(len(all_src_encodec))
print(len(all_instruction))
print(len(all_tgt_encodec))
eval_data_len = len(all_src_encodec) 

In [ ]:
def calculate_metrics(rewards):
    # Filter out None values
    valid_rewards = [r for r in rewards if r is not None]
    
    if len(valid_rewards) == 0:
        return {
            "mean": None,
            "median": None,
            "std_dev": None,
            "variance": None,
            "min": None,
            "max": None,
            "25th_percentile": None,
            "75th_percentile": None,
            "rewards": None,
        }
    
    metrics = {
        "mean": np.mean(valid_rewards),
        "median": np.median(valid_rewards),
        "std_dev": np.std(valid_rewards),
        "variance": np.var(valid_rewards),
        "min": np.min(valid_rewards),
        "max": np.max(valid_rewards),
        "25th_percentile": np.percentile(valid_rewards, 25),
        "75th_percentile": np.percentile(valid_rewards, 75),
        "rewards": rewards,
    }
    return metrics


def save_metrics(metrics, path,times, name):
    with open(f"{path}/{name}_{times}.json", "w") as f:
        json.dump(metrics, f, indent=4)

In [ ]:
model_name = "0704-1848"
output_dir_name = f"vc_model_stability_{model_name}"
temp = f"{base_path}/output/{output_dir_name}"
if not os.path.exists(temp):
    os.makedirs(temp)

output_path = f"{base_path}/vc_stability"
# if not os.path.exists(output_path):
#     os.makedirs(output_path)

nar_model, ar_tokenizer, nar_tokenizer = load_models_and_tokenizers(ar_checkpoint, nar_checkpoint)
args_predict, test_dataset = load_datasets(base_path, output_dir_name, device)
model = BartForConditionalGeneration.from_pretrained(ar_checkpoint, return_dict=True)
trained_model = BartForConditionalGeneration.from_pretrained(f"{base_path}/model_output/{model_name}/dpo_model")
model_reward = [] 

times = 10
num_src_encodec = 10

all_metrics = {}

for idx in range(num_src_encodec):
    src_encodec = all_src_encodec[idx]
    instruction = all_instruction[idx]
    model.to(device)
    model_reward = []

    for i in range(times):
        reward = process_and_get_scores(trained_model, nar_model, ar_tokenizer, nar_tokenizer, src_encodec, instruction, args_predict, episode_counter=i)
        model_reward.append(reward)

    metrics = calculate_metrics(model_reward)
    all_metrics[f"metrics_{idx}"] = metrics

save_metrics(all_metrics, output_path, times, output_dir_name)

In [ ]:
print(all_metrics)

# Temperature Test 

In [1]:
from dpo_eval import eval_dpo, save_metrics, prepare_data, load_models_and_tokenizers, load_datasets, process_and_get_scores
import torch
import sys
import importlib
import torch
from transformers import BartForConditionalGeneration, AutoTokenizer
import numpy as np
import json
import os

sys.path.append("/work/b0990106x/trl/vc")
import vc
importlib.reload(vc)


/home/b0990106x/miniconda3/envs/trl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'vc' (<_frozen_importlib_external._NamespaceLoader object at 0x7f3db5f62560>)>

In [2]:
ar_checkpoint = "lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans"
nar_checkpoint = "lca0503/speech-chatgpt-base-nar-v2-epoch4-wotrans"
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_data_len = 10
agent_input_dir = "/work/b0990106x/trl/data-encodec"
base_path = "/work/b0990106x/trl"

In [3]:
def load_from_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    all_src_encodec = [item["src_encodec"] for item in data]
    all_instruction = [item["instruction"] for item in data]
    all_tgt_encodec = [item["tgt_encodec"] for item in data]
    
    return all_src_encodec, all_instruction, all_tgt_encodec

# Usage
all_src_encodec, all_instruction, all_tgt_encodec = load_from_json('src_encodec.json')

print(len(all_src_encodec))
print(len(all_instruction))
print(len(all_tgt_encodec))
eval_data_len = len(all_src_encodec) 

9254
9254
9254


In [4]:
def calculate_metrics(rewards):
    # Filter out None values
    valid_rewards = [r for r in rewards if r is not None]
    
    if len(valid_rewards) == 0:
        return {
            "mean": None,
            "median": None,
            "std_dev": None,
            "variance": None,
            "min": None,
            "max": None,
            "25th_percentile": None,
            "75th_percentile": None,
            "rewards": None,
        }
    
    metrics = {
        "mean": np.mean(valid_rewards),
        "median": np.median(valid_rewards),
        "std_dev": np.std(valid_rewards),
        "variance": np.var(valid_rewards),
        "min": np.min(valid_rewards),
        "max": np.max(valid_rewards),
        "25th_percentile": np.percentile(valid_rewards, 25),
        "75th_percentile": np.percentile(valid_rewards, 75),
        "rewards": rewards,
    }
    return metrics


def save_metrics(metrics, path,times, name):
    with open(f"{path}/{name}_{times}.json", "w") as f:
        json.dump(metrics, f, indent=4)

In [5]:
model_name = "original_model_2"
output_dir_name = f"temp_test_{model_name}"
temp = f"{base_path}/output/{output_dir_name}"
if not os.path.exists(temp):
    os.makedirs(temp)

output_path = f"{base_path}/temp_test"
# if not os.path.exists(output_path):
#     os.makedirs(output_path)

nar_model, ar_tokenizer, nar_tokenizer = load_models_and_tokenizers(ar_checkpoint, nar_checkpoint)
args_predict, test_dataset = load_datasets(base_path, output_dir_name, device)
ar_model = BartForConditionalGeneration.from_pretrained(ar_checkpoint, return_dict=True)
# trained_model = BartForConditionalGeneration.from_pretrained(f"{base_path}/model_output/{model_name}/dpo_model")
temperature = 1.0
times = 10
num_src_encodec = 10

all_metrics = {}
# 1. select one src_encodec and instruction and then run inference 10 times to get 10 scores of the same temperature
# 2. Iterate through all the temperature from 0.0 to 2.0

temperature_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]


src_encodec = all_src_encodec[0]
instruction = all_instruction[0]

for temperature in temperature_list:
    ar_model.to(device)
    model_reward = []
    print(f"Temperature: {temperature}")
    for i in range(times):
        try:
            reward = process_and_get_scores(ar_model, nar_model, ar_tokenizer, nar_tokenizer, src_encodec, instruction, args_predict, episode_counter=0, temperature = temperature)
            model_reward.append(reward)
        except Exception as e:
            print("Error:", e)
            model_reward.append(None)

    metrics = calculate_metrics(model_reward)
    all_metrics[f"temperature_{temperature}"] = metrics

save_metrics(all_metrics, output_path, times, output_dir_name)


# for idx in range(num_src_encodec):
#     src_encodec = all_src_encodec[idx]
#     instruction = all_instruction[idx]
#     model.to(device)
#     model_reward = []

#     for i in range(times):
#         reward = process_and_get_scores(ar_model, nar_model, ar_tokenizer, nar_tokenizer, src_encodec, instruction, args_predict, episode_counter=i)
#         model_reward.append(reward)

#     metrics = calculate_metrics(model_reward)
#     all_metrics[f"metrics_{idx}"] = metrics

# save_metrics(all_metrics, output_path, times, output_dir_name)

/home/b0990106x/miniconda3/envs/trl/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Temperature: 0.1


/home/b0990106x/miniconda3/envs/trl/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Episode 0 : audio saved to  /work/b0990106x/trl/output/temp_test_original_model_2/example_save_0.wav


/home/b0990106x/miniconda3/envs/trl/lib/python3.10/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


...

Reward: 2.0964345932006836


/home/b0990106x/miniconda3/envs/trl/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Episode 0 : audio saved to  /work/b0990106x/trl/output/temp_test_original_model_2/example_save_0.wav


/home/b0990106x/miniconda3/envs/trl/lib/python3.10/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


Reward: 2.3229873180389404
